In [1]:
### Importing public functions
import os
import sys
import importlib
import json
import csv
from dataclasses import dataclass, field, fields
from typing import Any
from itertools import compress, product
from array import array
import datetime as dt
import time


import pandas as pd
import numpy as np

import sklearn as sk

import scipy
from scipy.ndimage import uniform_filter1d

import matplotlib.pyplot as plt

# import seaborn as sns

from scipy import signal

# import ghostipy as gsp
# from pacpy.pac import plv

# from scipy import fftpack, stats
# #mne
# import mne_bids
# import mne


In [6]:
def get_project_path_in_notebook(
    subfolder: str = '',
):
    """
    Finds path of projectfolder from Notebook.
    Start running this once to correctly find
    other modules/functions
    """
    path = os.getcwd()

    while path[-20:] != 'dyskinesia_neurophys':

        path = os.path.dirname(path)
    
    return path

In [7]:
# define local storage directories
projectpath = get_project_path_in_notebook()
codepath = os.path.join(projectpath, 'code')
figpath = os.path.join(projectpath, 'figures')
datapath = os.path.join(projectpath, 'data')


In [25]:
# import own functions
os.chdir(codepath)

import utils.utils_fileManagement as fileMng
import lfpecog_preproc.preproc_data_management as dataMng
import utils.utils_tmsi_poly5reader as read_poly5

In [137]:
def reref_neighbor_subtract(data, chnames):
    """
    Bipolar Rereferenceing against neighbouring contact
    
    Input:
        - data: 2d array [n-channels, n-samples]
        - chnames
    
    Returns:
        - new data [n-channels - 1, n-samples]
        - new channel names
    
    """

    data_out = []
    chnames_out = []

    for side in ['L', 'R']:

        side_chs = [ch for ch in chnames if f'LFP{side}' in ch]

        for i in range(len(side_chs) - 1):

            ch1, ch2 = side_chs[i], side_chs[i + 1]

            ich1 = np.where([ch == ch1 for ch in chnames])[0][0]
            ich2 = np.where([ch == ch2 for ch in chnames])[0][0]

            data_out.append(data[ich1] - data[ich2])
            chnames_out.append(f'LFP_{side}_0{ch1[4]}0{ch2[4]}')

    data_out = np.array(data_out)

    return data_out, chnames_out


#### find data

In [23]:
onedrive = os.path.abspath(
    os.path.join(fileMng.get_onedrive_path('onedrive'),
    os.pardir))
datadir = os.path.join(onedrive, 'Dokumente', 'data', 'dyst_pilot')

files = [f for f in os.listdir(datadir) if f[-5:] == 'Poly5']

In [36]:
files

['554NE65_MedOff02_Rest_StimOff_01-20221212T151318.DATA.Poly5',
 '554NE65_MedOff02_Rest_StimOff_02-20221212T151828.DATA.Poly5']

In [38]:
rest01 = os.path.join(datadir, files[0])
rest01 = read_poly5.Poly5Reader(rest01)

rest02 = os.path.join(datadir, files[1])
rest02 = read_poly5.Poly5Reader(rest02)

Reading file  c:\Users\habetsj\OneDrive - Charité - Universitätsmedizin Berlin\Dokumente\data\dyst_pilot\554NE65_MedOff02_Rest_StimOff_01-20221212T151318.DATA.Poly5
	 Number of samples:  1073760 
	 Number of channels:  34 
	 Sample rate: 4000 Hz
Done reading data.
Reading file  c:\Users\habetsj\OneDrive - Charité - Universitätsmedizin Berlin\Dokumente\data\dyst_pilot\554NE65_MedOff02_Rest_StimOff_02-20221212T151828.DATA.Poly5
	 Number of samples:  861120 
	 Number of channels:  34 
	 Sample rate: 4000 Hz
Done reading data.


In [43]:
data01 = rest01.read_data_MNE()
data02 = rest02.read_data_MNE()

Creating RawArray with float64 data, n_channels=34, n_times=1073760
    Range : 0 ... 1073759 =      0.000 ...   268.440 secs
Ready.


c:\Users\habetsj\Research\projects\dyskinesia_neurophys\code\utils\utils_tmsi_poly5reader.py:94: RuntimeWarning: Channel names are not unique, found duplicates for: {'X', 'Y', 'ISO aux', 'Z'}. Applying running numbers for duplicates.
  info = mne.create_info(ch_names=labels, sfreq=fs, ch_types=types_clean)
c:\Users\habetsj\Research\projects\dyskinesia_neurophys\code\utils\utils_tmsi_poly5reader.py:94: RuntimeWarning: Channel names are not unique, found duplicates for: {'X', 'Y', 'ISO aux', 'Z'}. Applying running numbers for duplicates.
  info = mne.create_info(ch_names=labels, sfreq=fs, ch_types=types_clean)


Creating RawArray with float64 data, n_channels=34, n_times=861120
    Range : 0 ... 861119 =      0.000 ...   215.280 secs
Ready.


In [75]:
from lfpecog_preproc import (
    preproc_filters,
    preproc_resample,
    preproc_reref
)

In [83]:
importlib.reload(preproc_resample)

<module 'lfpecog_preproc.preproc_resample' from 'c:\\Users\\habetsj\\Research\\projects\\dyskinesia_neurophys\\code\\lfpecog_preproc\\preproc_resample.py'>

In [139]:
ephys_data, ephys_ch = {}, {}
fs = int(data01.info['sfreq'])
fs_new = 2000

for temp, n in zip([data01, data02], ['01', '02']):

    ephys_ch_sel = ['LFP' in ch for ch in temp.ch_names]

    ephys_data[n] = temp.get_data()[ephys_ch_sel]
    ephys_ch[n] = list(compress(temp.ch_names, ephys_ch_sel))
    
    # bandpass filter
    ephys_data[n] = preproc_filters.bp_filter(
        ephys_data[n], n_timeRows=0, Fs=fs, 
        l_freq=2, h_freq=98, 
    )
    # notch filter
    ephys_data[n] = preproc_filters.notch_filter(
        ephys_data[n], n_timeRows=0, Fs=fs, 
        transBW=10, notchW=4, 
    )
    # resample
    ephys_data[n] = preproc_resample.resample(
        ephys_data[n], n_timeRows=0,
        Fs_orig=fs, Fs_new=fs_new, 
    )
    # rereference against neighbor
    ephys_data[n], ephys_ch[n] = reref_neighbor_subtract(
        ephys_data[n], ephys_ch[n])
    print( ephys_data[n].shape)

(14, 536880)
(14, 430560)


In [142]:
fig, axes = plt.subplots(2, 2, figsize=(18, 12))

for i_rest, n in enumerate(['01', '02']):

    for i_side, side in enumerate(['L', 'R']):

        for i_ch, ch in enumerate(ephys_ch[n]):

            if f'_{side}_' not in ch: continue

            f, psx = signal.welch(
                ephys_data[n][i_ch, 10 * fs_new:-10 * fs_new],
                fs=fs_new, nfft=int(fs_new / 2),
            )
            axes[i_rest, i_side].plot(f, psx, label=f'{ch}')


        axes[i_rest, i_side].legend(frameon=False)
        axes[i_rest, i_side].set_xlim(0, 50)
        axes[i_rest, i_side].set_title(f'Rest {n} - {side}')
plt.close()